In [15]:
#import
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import numpy as np
import math

In [11]:
# init Node class
class Node:
    def __init__(self, x_j, threshold, left, right, value):
        self.x_j = x_j
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  #if leaf  

In [ ]:
# init DecisionTree class
class DecisionTree:
    def __init__(self, max_depth, min_samples_split):
        self.max_depth = max_depth
        self.min_samples_split= min_samples_split
        self.root = None

    def fit(self, x, y):

    def predict(self, x):

    def build_tree(self, x, y, depth):

In [ ]:
#loads dataset
cancer = load_breast_cancer() 
# tot samples = 569
# 212 malignant, 357 benign
# 30 features/columns

x = pd.DataFrame(cancer.data, columns = cancer.feature_names)
y = pd.Series(cancer.target)

print("Target class distribution:")
print(y.value_counts())

Target class distribution:
1    357
0    212
Name: count, dtype: int64


In [10]:
# split dataset too 70% training, 15% testing, 15% validation

x_train, x_temp, y_train, y_temp = train_test_split(
    x, y,
    test_size = 0.3,
    random_state = 42,
    stratify = y
)

x_test, x_validation, y_test, y_validation = train_test_split(
    x_temp, y_temp,
    test_size = 0.5,
    random_state = 42,
    stratify = y_temp
)

print(f"Training set size: {x_train.shape[0]} samples ({round(x_train.shape[0]/x.shape[0]*100)}%)")
print(f"Temporary set size: {x_temp.shape[0]} samples ({round(x_temp.shape[0]/x.shape[0]*100)}%)")
print(f"Validation set size: {x_validation.shape[0]} samples ({round(x_validation.shape[0]/x.shape[0]*100)}%)")
print(f"Test set size: {x_test.shape[0]} samples ({round(x_test.shape[0]/x.shape[0]*100)}%)")

Training set size: 398 samples (70%)
Temporary set size: 171 samples (30%)
Validation set size: 86 samples (15%)
Test set size: 85 samples (15%)


In [18]:
def calculate_entropy(y):
    entropy = 0
    totSamples = len(y)
    # get unique class labels and their counts
    # unique returns the sorted unique elements
    # counts returns the number of times each unique element appears
    class_labels, counts = np.unique(y, return_counts=True)
    
    for i in counts:
        p_k = i / totSamples
        entropy += (-p_k * math.log2(p_k))

    return entropy

In [19]:
def find_best_split(x, y):
    max_info_gain = -math.inf
    best_feature = None
    best_threshold = None
    totSamples = len(y)

    entropy_Y = calculate_entropy(y)

    for feature in range(x.shape[1]):

        feature_data = x.iloc[:, feature].unique()
        sorted_unique_values = np.sort(feature_data)

        for i in range(len(sorted_unique_values) - 1):

            candidate_threshold = (sorted_unique_values[i] + sorted_unique_values[i + 1]) / 2

            filter_left = feature_data <= candidate_threshold
            filter_right = feature_data > candidate_threshold

            y_left = y[filter_left]
            y_right = y[filter_right]

            samples_left = len(y_left)
            samples_right = len(y_right)

            #avoids splits where one child node is empty
            if samples_left == 0 or samples_right == 0:
                continue

            entropy_left = calculate_entropy(y_left)
            entropy_right = calculate_entropy(y_right)

            entropy_YgivenX = (samples_left / totSamples) * entropy_left + (samples_right / totSamples) * entropy_right

            info_gain = entropy_Y - entropy_YgivenX

            if info_gain > best_gain:
                best_gain = info_gain
                best_feature = feature
                best_threshold = candidate_threshold

    return max_info_gain, best_feature, best_threshold